In [2]:
import chess
import chess.pgn
import os

In [2]:
def read_games_from_file(file_path):
    games = []
    with open(file_path, 'r') as pgn_file:
        while True:
            try:
                game = chess.pgn.read_game(pgn_file)
                if game is None:
                    break
                games.append(game)
            except ValueError as e:
                print(f"Pominięto partię z powodu błędu: {e}")
    return games

def is_checkmate(game):
    board = game.board()
    for move in game.mainline_moves():
        board.push(move)
    return board.is_checkmate()

def filter_checkmate_games(games, pgn_mate_path, pgn_draw_path):
    checkmate_games = []
    draw_games = []
    for cnt, game in enumerate(games):
        try:
            res = is_checkmate(game)
            if res:
                checkmate_games.append(game)
            elif game.headers["Result"] == "1/2-1/2":
                draw_games.append(game)
        except Exception as e:
            print(f"Pominięto partię numer {cnt} z powodu błędu: {e}")

    with open(pgn_mate_path, "a") as output_file:
        for game in checkmate_games:
            output_file.write(str(game))
            output_file.write("\n\n")

    with open(pgn_draw_path, "a") as output_file:
        for game in draw_games:
            output_file.write(str(game))
            output_file.write("\n\n")


data_path = "../data/raw/"

files = os.listdir(data_path)

for file in files:
    games = read_games_from_file(data_path + file)
    filter_checkmate_games(games, data_path + "mates.pgn", data_path + "draw.pgn")

illegal san: 'b1' in rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq - 0 1 while parsing <Game at 0x7d5c0551b8b0 ('?' vs. '?', '????.??.??' at '?')>


In [4]:
games = read_games_from_file(data_path+"mates.pgn")
drawn = read_games_from_file(data_path + "draw.pgn")

In [6]:


invalid = 0
for cnt, game in enumerate(drawn):
    data = []
    board = chess.Board()
    cor = True
    

    for move in game.mainline_moves():
        legal_moves = list(board.legal_moves)
        if move in legal_moves:
            board_fen = board.fen()
            board_fen += "5-5"+"\n"
            data.append(board_fen)
            board.push(move)
        else:
            cor = False
            invalid += 1
            break
    if cor:
        with open(data_path+"drawn.txt", "a") as f:
            f.writelines(data)
    if cnt % 1000 == 0:
        print(f"{cnt}/{len(drawn)}")

        
print("invalid drawns: ", invalid)
invalid = 0

for cnt, game in enumerate(games):
    data = []
    board = chess.Board()
    cor = True
    

    for move in game.mainline_moves():
        legal_moves = list(board.legal_moves)
        if move in legal_moves:
            board_fen = board.fen()
            board_fen += game.headers['Result'] + "\n"
            data.append(board_fen)
            board.push(move)
        else:
            cor = False
            invalid += 1
            break
    if cor:
        with open(data_path+"mates.txt", "a") as f:
            f.writelines(data)
    if cnt % 1000 == 0:
        print(f"{cnt}/{len(games)}")
print("invalid: ", invalid)

0/82977
1000/82977
2000/82977
3000/82977
4000/82977
5000/82977
6000/82977
7000/82977
8000/82977
9000/82977
10000/82977
11000/82977
12000/82977
13000/82977
14000/82977
15000/82977
16000/82977
17000/82977
18000/82977
19000/82977
20000/82977
21000/82977
22000/82977
23000/82977
24000/82977
25000/82977
26000/82977
27000/82977
28000/82977
29000/82977
30000/82977
31000/82977
32000/82977
33000/82977
34000/82977
35000/82977
36000/82977
37000/82977
38000/82977
39000/82977
40000/82977
41000/82977
42000/82977
43000/82977
44000/82977
45000/82977
46000/82977
47000/82977
48000/82977
49000/82977
50000/82977
51000/82977
52000/82977
53000/82977
54000/82977
55000/82977
56000/82977
57000/82977
58000/82977
59000/82977
60000/82977
61000/82977
62000/82977
63000/82977
64000/82977
65000/82977
66000/82977
67000/82977
68000/82977
69000/82977
70000/82977
71000/82977
72000/82977
73000/82977
74000/82977
75000/82977
76000/82977
77000/82977
78000/82977
79000/82977
80000/82977
81000/82977
82000/82977
invalid drawns:  

In [4]:
import torch
data_path = "../data/raw/"
def create_tensor(board: chess.Board, transform: bool=False):
    matrix_board = torch.zeros((6, 8, 8))
    for i in range(8):
        for j in range(8):
            piece = board.piece_at(chess.square(i, j))
            if piece is not None:
                piece_type = piece.piece_type
                piece_color = piece.color
                index = piece_type - 1

                row = 7-j if not transform else j

                if piece_color == chess.WHITE:
                    matrix_board[index, row, i] = 1
                else:
                    matrix_board[index, row, i] = -1
    if transform:
        matrix_board *= -1
        matrix_board = torch.where(torch.abs(matrix_board) < 1e-6, torch.zeros_like(matrix_board), matrix_board)
    return matrix_board


tensors = []
labels = []
with open(data_path+"mates.txt", "r") as f:
    data = f.readlines()

white_win = 0
black_win = 0
for line in data:
    fen, result = line[:-4], line[-4:]
    if result[0] == "1":
        white_win += 1
    else:
        black_win += 1
    tensor = create_tensor(chess.Board(fen))
    label = int(result[0])
    tensors.append(tensor)
    labels.append(label)




In [6]:
print(f"black win: {black_win}")
print(f"white win: {white_win}")

draw_num = int((black_win + white_win) / 2)

with open(data_path+"drawn.txt", "r") as f:
    data = f.readlines()

import random

random.shuffle(data)

data = data[:draw_num]

labels.extend([0.5 for _ in range(len(data))])

for line in data:
    fen = line[:-4]
    tensor = create_tensor(chess.Board(fen))
    tensors.append(tensor)


black win: 3311718
white win: 4450445


In [8]:
x = torch.stack(tensors)
y = torch.tensor(labels)

torch.save(x, "../data/prep/X.pt")
torch.save(y, "../data/prep/y.pt")